In [97]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

In [98]:
trained_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(224,224,3), include_top=False, weights="imagenet")

In [99]:
trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_282 (Batch  (None, 111, 111, 32  96         ['conv2d_282[0][0]']             
 Normalization)                 )                                                      

In [100]:
for layer in trained_model.layers:
  layer.trainable = False
last_layer = trained_model.get_layer('mixed7')
print(last_layer.output_shape)
last_output = last_layer.output

(None, 12, 12, 768)


In [101]:
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)
model = Model(trained_model.input, x)
model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = tf.keras.losses.MeanSquaredError(),
              metrics = ['accuracy'])

In [102]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_282 (Batch  (None, 111, 111, 32  96         ['conv2d_282[0][0]']             
 Normalization)                 )                                                           

In [104]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [105]:
base = '/content/drive/MyDrive/data'
train = os.path.join(base,'Train')
test = os.path.join(base,'Test')

In [107]:
train_settings = ImageDataGenerator(rescale=1.0/255.,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
test_settings = ImageDataGenerator(rescale=1.0/255.)

In [108]:
train_generator = train_settings.flow_from_directory(train,
                                                    batch_size = 64,
                                                    class_mode = 'binary',
                                                    target_size=(224,224))
test_generator = test_settings.flow_from_directory(test,
                                                  batch_size = 20,
                                                  class_mode = 'binary',
                                                  target_size=(224,224))

Found 2342 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [109]:
history = model.fit(train_generator,
                    validation_data=test_generator,
                    steps_per_epoch=12,
                    epochs=5,
                    validation_steps=3,
                    verbose=1)

Epoch 1/5
12/12 [==============================] - 68s 5s/step - loss: 0.3699 - accuracy: 0.5957 - val_loss: 0.0119 - val_accuracy: 0.9833
Epoch 2/5
12/12 [==============================] - 67s 5s/step - loss: 0.0368 - accuracy: 0.9531 - val_loss: 3.2324e-05 - val_accuracy: 1.0000
Epoch 3/5
12/12 [==============================] - 65s 5s/step - loss: 0.0194 - accuracy: 0.9740 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 64s 5s/step - loss: 0.0386 - accuracy: 0.9570 - val_loss: 7.3025e-05 - val_accuracy: 1.0000
Epoch 5/5
12/12 [==============================] - 63s 5s/step - loss: 0.0210 - accuracy: 0.9704 - val_loss: 4.6456e-06 - val_accuracy: 1.0000


In [110]:
print(history.history['accuracy'])

[0.5956873297691345, 0.953125, 0.9739583134651184, 0.95703125, 0.9703503847122192]


In [111]:
model.save('/content/drive/MyDrive/data/maskdetector.h5')

Test

In [116]:
import cv2
import keras
testimg = '/content/2022-02-26-092008 (1).jpg'
pic = cv2.imread(testimg)
pic = cv2.resize(pic,(224,224))
pic = pic.reshape(1,224,224,3)
data = tf.keras.applications.inception_v3.preprocess_input(pic,data_format='channels_last')
print(data.shape)

(1, 224, 224, 3)


In [113]:
maskdetector = keras.models.load_model('/content/drive/MyDrive/data/maskdetector.h5')

if predictions closer to 1 unmasked else masked

In [117]:
predictions = maskdetector.predict(data, verbose=1)

1/1 [==============================] - 0s 108ms/step


In [118]:
print(predictions)

[[0.9761262]]
